In [1]:
from neo4j import GraphDatabase
import pandas as pd

### **Database connection**

Here, the connection settings to link this script with Neo4j are setup

In [5]:
# Connection settings
uri = "neo4j://localhost:7687"
pass = "pass"
driver = GraphDatabase.driver(uri, auth=("neo4j", pass))

### **Data import**

Import the new needed data to upload

In [6]:
# Data import
# Nodes
organizations = pd.read_csv('./data/organizations.csv')

# Relations
author_reviewerGroup = pd.read_csv('./data/auth_is_reviewerVersion2.csv')
# contains the respective review and a veredict flag
author_organization = pd.read_csv('./data/author_organization.csv')



### **Data Loading**

In this stage we load the data 

#### **Nodes**

*Organizations:*

In [7]:
# Implementation
def create_organizations(tx, organizations):
    for i in range(len(organizations)):
        tx.run("MERGE (:Organization {organization_id: $organization_id, type: $type, name: $name})", organization_id=int(organizations.loc[i, "organization:ID"]), type=organizations.loc[i, "type:string"], name=organizations.loc[i, "name:string[]"])

# Invocation
with driver.session() as session:
    session.write_transaction(create_organizations, organizations)

#### **Edges**

*Author-Organization*

In [8]:
# Implementation
def create_affiliated_to(tx, author_organization):
    for i in range(len(author_organization)):
        tx.run("MATCH (n: Author {author_id: $start}), (m: Organization {organization_id: $end}) MERGE (n) - [:affiliated_to] -> (m) ", start=int(author_organization.loc[i, ":START_ID"]), end=int(author_organization .loc[i, ":END_ID"]))

# Invocation
with driver.session() as session:
    session.write_transaction(create_affiliated_to, author_organization)

*Author-ReviewerGroup*

This edge already existed, but with the new requirements, it has to carry some additional information in it's properties: the review text and the review veredict of the author for the corresponding article. As such, this edge will be updated with the corresponding new properties.

In [9]:
# Implementation
def update_reviewer_in(tx, author_reviewerGroup):
    for i in range(len(author_reviewerGroup)):
        tx.run("MATCH (n: Author {author_id: $start}), (m: ReviewGroup {assignedReviewer_id: $end}) MERGE (n) - [:reviewer_in {review_details: $review, decision: $decision }] -> (m) ", 
        start=int(author_reviewerGroup.loc[i, ":START_ID"]), 
        end=int(author_reviewerGroup .loc[i, ":END_ID"]),
        decision=int(author_reviewerGroup.loc[i, "decision:bool"]), 
        review=author_reviewerGroup .loc[i, "text:strin[]"])

# Invocation
with driver.session() as session:
    session.write_transaction(update_reviewer_in, author_reviewerGroup)